Importing Libraries

In [89]:
import nltk
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk import FreqDist, pos_tag
from wordcloud import WordCloud
from nltk.corpus import stopwords
import re
import numpy as np
from nltk.corpus import wordnet as wn
from collections import Counter
import spacy
from spacy import displacy
from collections import Counter
import pandas as pd


In [90]:
 nlp = spacy.load("en_core_web_sm")

book_used = 1

lines = ""
with open(f'/content/theScam.txt') as f:
    lines = f.readlines()
text = ""
for line in lines:
  text += line

Recognising Entities

In [91]:
def recognize_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

Splitting Chapter Wise Text

In [92]:

def process_book(book_path):
    with open(book_path, 'r', encoding='utf-8') as file:
        book_text = file.read()

    # Split the book into chapters (you might need a more sophisticated approach)
    chapters = book_text.split("CHAPTER")

    # Recognize entities in each chapter
    all_entities = []
    for chapter_number, chapter_text in enumerate(chapters):
        chapter_entities = recognize_entities(chapter_text)
        all_entities.extend([(entity[0], entity[1], chapter_number + 1) for entity in chapter_entities])

    return all_entities

Reading Book

In [93]:
# Path to your book file
book_path = '/content/theScam.txt'


In [94]:
# Get all entities and their types from the book
all_entities = process_book(book_path)

Printing All Entities

In [95]:
# Print the result
for entity, entity_type, chapter_number in all_entities:
    print(f"Entity: {entity}, Type: {entity_type}")

Streaming output truncated to the last 5000 lines.
Entity: 0.005%, Type: PERCENT
Entity: Standard, Type: ORG
Entity: Chartered Bank, Type: ORG
Entity: Canbank Financial Services, Type: ORG
Entity: Andhra Bank, Type: ORG
Entity: BR, Type: PERSON
Entity: Kurias, Type: PERSON
Entity: Andhra Bank, Type: ORG
Entity: VBD, Type: ORG
Entity: the Bank of Karad, Type: ORG
Entity: Metropolitan Cooperative Bank, Type: ORG
Entity: Stanchart, Type: PERSON
Entity: Kurias, Type: PERSON
Entity: Andhra Bank, Type: ORG
Entity: VBD, Type: ORG
Entity: one, Type: CARDINAL
Entity: RBI, Type: ORG
Entity: 1992, Type: DATE
Entity: Andhra Bank, Type: ORG
Entity: Shroff, Type: PERSON
Entity: Bank of Karad, Type: ORG
Entity: Allahabad Bank, Type: ORG
Entity: SLR, Type: ORG
Entity: RBI, Type: ORG
Entity: BR, Type: PERSON
Entity: Kurias, Type: PERSON
Entity: Syndicate Bank, Type: ORG
Entity: Andhra Bank, Type: ORG
Entity: Rs 15, Type: PRODUCT
Entity: Rs 97.75, Type: PRODUCT
Entity: 31, Type: CARDINAL
Entity: Decembe

Manual text from the book

In [96]:
text1 = """American Express did it for Ajay Kayan and Andhra Bank for
Hiten Dalal. Banks, very simply, were too keen on accommodating brokers.
This was manifested in many other ways – most importantly in the way
brokers were using bank money to do their own deals. And Harshad was not
alone. Banks would buy securities from the broker with the understanding
that the broker would take them back after a few months. In 1987 came the biggest incentive for corruption in the equity markets:
the mutual funds. Till then there was just one, the UTI headed by the eternal
optimist, MJ Pherwani, the original Big Bull. UTI, with a corpus of Rs
20,000 crore was the big dad of the market. If it bought, the market went up.
If it sold, a minor crash followed. Then, in 1987, the government-owned
banks and insurance companies were allowed to set up mutual funds, a
decision that was piloted by S Venkitaramanan when he was the finance
secretary."""


text2 = """CBI has filed seventy-two charge-sheets for criminal offences, of which
only four have gone anywhere. CBI quietly abandoned several important
trails. Investigation into Goldstar Steel, which involved Prime Minister
Narasimha Rao’s sons and Hiten Dalal is in limbo, as is the case against
former union minister B Shankaranand who was one of the two ministers
who resigned in the aftermath of the scam.
Of the cases that have come to some conclusion, the first, convicted Hiten
Dalal for bouncing of cheques that Stanchart had sent for clearing five"""


text3 = """There was more to the nexus. Pradip Dharandharkar, a chartered
accountant, and KN Vyas were co-opted to the MCB board on 19 th March
1991. Both were managers in Killick Nixon. Dharandharkar quit late in 1992
while Vyas continued to work in the export depart-ment. Another key
suspect was the chairman of the Metro Banks, Hemant Vyas, about whom
The Indian Express , in a report of 31 st May 1992, said that he was about to
be booked under the Conservation of Foreign Exchange and Prevention of
Smuggling Act in 1987-90. Express alleged that he was rescued by the finance
minister, ND Tiwari, whom he calls Chacha Tiwari. The charge against Vyas
was that he was smuggling polyester yarn. Neither Tiwari nor Vyas denied
the story. Vyas pleaded ignorance about the decisions taken by the MCB
board.In the end, JPC did not add anything to our knowledge of the scam and
exonerated several key players, by selective identification. It named Ketan
Parekh as the key player in the scam, gave Yashwant Sinha, the then finance
minister, a clean chit, falsely hung former finance secretary Ajit Kumar for
the UTI collapse and blamed SEBI and finance ministry for not being
vigilant. It also pointed to the nexus between Dinesh Dalmia of DSQ
Software, broker-directors of CSE, officials of Stock Holding Corporation
and UTI. It washed its hands off the scam by suggesting that SEBI or the
DCA should further investigate the nexus between corporate bodies and
brokers."""


Reading book text

In [97]:
# Function to read a book and recognize entities in all chapters
texts = [text1, text2, text3]

for text in texts:
    doc = nlp(text)

Manual Labelling

In [98]:
manual_labels1 = [("Ajay Kayan", "PERSON"), ("Hiten Dalal", "PERSON"), ("American Express", "ORG"), ("UTI", "ORG"), (" S Venkitaramanan", "PERSON")  ]
manual_labels2 = [("Narasimha Rao", "PERSON"), ("CBI", "ORG"), ("B Shankaranand", "PERSON"), ("Hiten Dalal", "PERSON")]
manual_labels3 = [("Killick Nixon", "PERSON"), ("Indian Express", "ORG"), ("Dinesh Dalmia", "PERSON"), ("Ketan Parekh", "FAC"), ("SEBI", "ORG") ]

Calculating Precision,Recall & F1 Score

In [99]:
# Calculate precision, recall, and F1 score
def calculate_metrics(predicted, actual):
    true_positives = len(set(predicted) & set(actual))
    false_positives = len(set(predicted) - set(actual))
    false_negatives = len(set(actual) - set(predicted))

    precision = true_positives / (true_positives + false_positives + 1e-9)if (true_positives + false_positives) != 0 else 0
    recall = true_positives / (true_positives + false_negatives + 1e-9)if (true_positives + false_positives) != 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-9)if (precision + recall) != 0 else 0

    return precision, recall, f1_score

In [100]:
doc1 = nlp(text1)
doc2 = nlp(text2)
doc3 = nlp(text3)
entities1 = [(ent.text, ent.label_) for ent in doc1.ents]
entities2 = [(ent.text, ent.label_) for ent in doc2.ents]
entities3 = [(ent.text, ent.label_) for ent in doc3.ents]

Calculating using Confusion Matrix

In [101]:
# Calculate metrics
precision, recall, f1_score = calculate_metrics(entities1,manual_labels1)
df1 = pd.DataFrame({'Text': [1], 'Precision': precision, 'Recall': recall, 'F1 Score': f1_score})
precision, recall, f1_score = calculate_metrics(entities2,manual_labels2)
df2 = pd.DataFrame({'Text': [2], 'Precision': precision, 'Recall': recall, 'F1 Score': f1_score})
precision, recall, f1_score = calculate_metrics(entities3,manual_labels3)
df3 = pd.DataFrame({'Text': [3], 'Precision': precision, 'Recall': recall, 'F1 Score': f1_score})

Printing Values of F_Score

In [104]:
# List of DataFrames
dfs = [df1, df2, df3]

# Combine DataFrames vertically
rdf = pd.concat(dfs, ignore_index=True)

# Print metrics
rdf

,Text,Precision,Recall,F1 Score
0,1,0.250000,0.60,0.352941
1,2,0.250000,0.75,0.375000
2,3,0.038462,0.20,0.064516
